# Lecture 2 - 2

We are going to use data on NYC (Paris) subway system to find the shortest path from 2 stations. The nodes are the subway station, stored in `nodes.csv` and the arcs are the distance / time between connecting nodes, stored in `arcs.csv`.

In [ ]:
library("gurobi")
library("Matrix")
library("magick")
city = "NYC"  # 'Paris'

Let's load up and examine our data

In [ ]:
thePath = getwd()
arcs = as.matrix(read.csv(paste0(thePath, "/", city, "/arcs.csv"), sep = ",", header = TRUE))  # loads the arc data
nodes = as.matrix(read.csv(paste0(thePath, "/", city, "/nodes.csv"), sep = ",", header = TRUE))  # loads the nodes data
head(arcs)

In [ ]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 20)  # to show the entire matrix (defaults are 60 and 20) so you can find your stops
nodes

Now let's get things into a form that we can use! The problem that we are trying to solve is 
\begin{align}
\min_{\pi \geq 0} \sum_{(x,y) \in \mathcal{A}} \pi_{xy} c_{xy} \\
\text{s.t. } \nabla^T \pi = n 
\end{align}

Suppose that the network has `nbNodes` nodes and `nbArcs` arcs.
* $\pi$ is our object of interest,
* $c$ a vector of length `nbArcs` such that $c_a$ is the transportation cost at arc $a$,
* $\nabla$ is an matrix of size `nbArcs` $\times$ `nbNodes`. If the first arc has $i$ as the origin node and $j$ as the the destination node then $\nabla_{1i} = -1$ and $\nabla_{1j} = 1$,
* $n$, a vector of length `nbNodes` such that $n_x$ is the net demand at node $x$, i.e. $n_i = -1$, $n_j = 1$;

In [ ]:
originNode = 383  # Union Sq. on the L train
destinationNode = 394  # Myrtle Wyckoff on the L train

In [ ]:
nbNodes = max(as.numeric(arcs[, 1]))
nbArcs = dim(arcs)[1]
namesNodes = nodes[, 1]
c = arcs[, 3]
n = rep(0, nbNodes)  # construct vector of exiting flow, net demand is zero
n[c(originNode, destinationNode)] = c(-1, 1)  # except for our origin and destination

In [ ]:
?sparseMatrix

In [ ]:
Nabla = sparseMatrix(i = 1:nbArcs, j = as.numeric(arcs[, 1]), dims = c(nbArcs, nbNodes), 
    x = -1) + sparseMatrix(i = 1:nbArcs, j = as.numeric(arcs[, 2]), dims = c(nbArcs, 
    nbNodes), x = 1)

In [ ]:
result = gurobi(list(A = t(Nabla), obj = as.numeric(c), sense = "=", rhs = n, modelsense = "min", 
    start = matrix(0, nbArcs, 1)), params = NULL)
pi = result$x
distance = result$objval

Let's deduce the minimal distance path

In [ ]:
# Some plotting stuff
themargin = -c(1, 1, 0.5, 0.2)
require("igraph")
geoCoordinates = nodes[, 3:4]
class(geoCoordinates) = "numeric"
# mapCoordinates = nodes[,5:6] class(mapCoordinates)='numeric'
nbNodes = max(arcs[, 1])
nbArcs = dim(arcs)[1]

plotCurrentNetwork = function(network, curNode) {
    sizeNodes = rep(1, nbNodes)
    sizeNodes[originNode] = 4
    sizeNodes[destinationNode] = 4
    sizeNodes[curNode] = 4
    labelNodes = rep(NA, nbNodes)
    labelNodes[originNode] = namesNodes[originNode]
    labelNodes[destinationNode] = namesNodes[destinationNode]
    labelNodes[curNode] = namesNodes[curNode]
    plot.igraph(network, vertex.label = labelNodes, vertex.label.cex = 1, vertex.size = sizeNodes, 
        edge.arrow.size = 0, layout = geoCoordinates, margin = themargin)
}

thegraph = graph_from_edgelist(arcs[, 1:2])

labelColors = rep("SkyBlue2", nbNodes)
labelColors[originNode] = "firebrick2"
labelColors[destinationNode] = "forestgreen"

sizeNodes = rep(1, nbNodes)
sizeNodes[originNode] = 4
sizeNodes[destinationNode] = 4

nbNodesSoFar = 1
curpoint = originNode

cont = TRUE
i = originNode
writeLines(paste0(namesNodes[i], " (#", i, ")"))
eqpath = which(pi > 0)
rank = 0

frames = image_graph(width = 600, height = 600, res = 150)

cont = TRUE
i = originNode
writeLines(paste0(namesNodes[i], " (#", i, ")"))
eqpath = which(pi > 0)
rank = 0
while (cont) {
    rank = rank + 1
    leavingi = which(Nabla[, i] == -1)
    a = intersect(eqpath, leavingi)[1]
    j = which(Nabla[a, ] == 1)[1]
    plotCurrentNetwork(thegraph, j)
    writeLines(paste0(rank, ": ", namesNodes[j], " (#", j, ")"))
    i = j
    if (j == destinationNode) {
        cont <- FALSE
    }
}
# done with plotting
dev.off()

In [ ]:
# animate
image_animate(frames, 1)